# Extracting and Visualizing Stock Data

In [1]:
# Downloading packages

!pip install yfinance==0.1.67
!mamba install bs4==4.10.0 -y

     |████████████████████████████████| 6.4 MB 6.0 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
/bin/bash: mamba: command not found


In [2]:
# Loading packages

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define Graphing Function

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Use yfinance to Extract Stock Data of Tesla

In [5]:
Tesla = yf.Ticker("TSLA")
tesla_data = Tesla.history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,3.800,5.000,3.508,4.778,93831500,0,0.0
1,2010-06-30,5.158,6.084,4.660,4.766,85935500,0,0.0
2,2010-07-01,5.000,5.184,4.054,4.392,41094000,0,0.0
3,2010-07-02,4.600,4.620,3.742,3.840,25699000,0,0.0
4,2010-07-06,4.000,4.000,3.166,3.222,34334500,0,0.0


## Use Webscraping to Extract Tesla Revenue Data

In [7]:
url = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'
html_data = requests.get(url).text

soup = BeautifulSoup(html_data, 'html.parser')

In [9]:
tables = soup.find_all('table')

for index, table in enumerate(tables):
    if('Tesla Quarterly Revenue') in str(table):
        table_index = index
        
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in tables[table_index].tbody.find_all('tr'):
    col = row.find_all('td')
    
    if(col != []):
        date = col[0].text
        revenue = col[1].text
        
        tesla_revenue = tesla_revenue.append({'Date': date, 'Revenue': revenue}, ignore_index = True)
        
tesla_revenue.head()

,Date,Revenue
0,2021-12-31,"$17,719"
1,2021-09-30,"$13,757"
2,2021-06-30,"$11,958"
3,2021-03-31,"$10,389"
4,2020-12-31,"$10,744"


In [10]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

tesla_revenue.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,Date,Revenue
45,2010-09-30,31
46,2010-06-30,28
47,2010-03-31,21
49,2009-09-30,46
50,2009-06-30,27


## Use yfinance to Extract Stock Data of Game Stop

In [11]:
GameStop = yf.Ticker("GME")

gme_data = GameStop.history(period="max")

gme_data.reset_index(inplace = True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13,6.480511,6.773397,6.413181,6.766664,19054000,0.0,0.0
1,2002-02-14,6.850828,6.864294,6.682503,6.733001,2755400,0.0,0.0
2,2002-02-15,6.733000,6.749832,6.632005,6.699335,2097400,0.0,0.0
3,2002-02-19,6.665670,6.665670,6.312187,6.430015,1852600,0.0,0.0
4,2002-02-20,6.463682,6.648839,6.413184,6.648839,1723200,0.0,0.0


## Use Webscraping to Extract GME Revenue Data

In [12]:
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_data = requests.get(url).text

soup = BeautifulSoup(html_data, 'html5lib')

In [13]:
tables = soup.find_all('table')

for index, table in enumerate(tables):
    if('GameStop Quarterly Revenue') in str(table):
        table_index = index
        
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in tables[table_index].tbody.find_all('tr'):
    col = row.find_all('td')
    if(col != []):
        date = col[0].text
        revenue = col[1].text.replace('$', '').replace(',','')
        gme_revenue = gme_revenue.append({"Date": date, "Revenue": revenue}, ignore_index = True)
        
gme_revenue.head()

,Date,Revenue
0,2021-10-31,1297
1,2021-07-31,1183
2,2021-04-30,1277
3,2021-01-31,2122
4,2020-10-31,1005


In [14]:
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")

gme_revenue.dropna(inplace=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

gme_revenue.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,Date,Revenue
47,2010-01-31,3524
48,2009-10-31,1835
49,2009-07-31,1739
50,2009-04-30,1981
51,2009-01-31,3492


## Using the yfinance Library to Extract Stock Data of Apple

In [15]:
apple = yf.Ticker("AAPL")

apple_share_price_data = apple.history(period="max")

apple_share_price_data.reset_index(inplace=True)

apple_share_price_data.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
10393,2022-03-03,168.470001,168.910004,165.550003,166.229996,76678400,0.0,0.0
10394,2022-03-04,164.490005,165.550003,162.100006,163.169998,83737200,0.0,0.0
10395,2022-03-07,163.360001,165.020004,159.039993,159.300003,96418800,0.0,0.0
10396,2022-03-08,158.820007,162.880005,155.800003,157.440002,131148300,0.0,0.0
10397,2022-03-09,161.479996,163.410004,159.410004,162.949997,91281100,0.0,0.0


## Use Webscraping to Extract Apple Revenue Data

In [16]:
url = "https://www.macrotrends.net/stocks/charts/AAPL/apple/revenue"
html_data = requests.get(url).text

soup = BeautifulSoup(html_data, 'html5lib')

In [17]:
tables = soup.find_all('table')

for index, table in enumerate(tables):
    if('Apple Quarterly Revenue') in str(table):
        table_index = index
        
apple_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in tables[table_index].tbody.find_all('tr'):
    col = row.find_all('td')
    if(col != []):
        date = col[0].text
        revenue = col[1].text.replace('$', '').replace(',','')
        apple_revenue = apple_revenue.append({"Date": date, "Revenue": revenue}, ignore_index = True)
        
apple_revenue.head()

,Date,Revenue
0,2021-12-31,123945
1,2021-09-30,83360
2,2021-06-30,81434
3,2021-03-31,89584
4,2020-12-31,111439


## Plot Tesla Stock Graph

In [18]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

## Plot GameStop Stock Graph

In [20]:
make_graph(gme_data, gme_revenue, 'GameStop')

## Plot Apple Stock Graph

In [22]:
make_graph(apple_share_price_data, apple_revenue, 'Apple')